<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_threads_Cpp_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

Solución implementada utilizando un *aproach* funcional, priorizando la inmutabilidad de los datos

In [23]:
%%writefile matrix_mult.cpp
#include <iostream>
#include <vector>
#include <string>
#include <random>
#include <utility>
#include <thread>
#include <stdlib.h>

#define MIN_MATRIX_SIZE 5
#define MAX_MATRIX_SIZE 20
#define MIN_MATRIX_CELL_VALUE -32
#define MAX_MATRIX_CELL_VALUE 32

// Definimos a la matriz como un tipo de datos vector de vector de t_matrix
// t_matrix es el tipo de dato que contiene la matriz
// está definido en minuscula para no conufndir con los tipos de datos genéricos ej <T>
typedef int t_matrix;
typedef std::vector<std::vector<t_matrix>> Matrix;

int validateArgs(int& argc, char* argv[]);
Matrix matrixGenerator(int size, bool populate = true);
Matrix matrixGenerator(int row_size, int col_size, bool populate = true);
Matrix secuentialMult(const Matrix&, const Matrix&);
Matrix concurrentMult(const Matrix&, const Matrix&);
void printMatrix(const std::string alias, const Matrix&);

int main(int argc, char* argv[])
{
    int size = validateArgs(argc, argv);

    Matrix matrix_a  {matrixGenerator(size)};
    Matrix matrix_b  {matrixGenerator(size)};
    Matrix matrix_cs {secuentialMult(matrix_a, matrix_b)};
    Matrix matrix_ch {concurrentMult(matrix_a, matrix_b)};

    printMatrix("A", matrix_a);
    printMatrix("B", matrix_b);
    printMatrix("CS", matrix_cs);
    printMatrix("CH", matrix_ch);
    std::cout << "Las matrices CS y CH son iguales? "
        // El operador == en vectores compara todos los elementos del vector
        << (matrix_cs == matrix_ch ? "Si" : "No")
        << std::endl;

    return EXIT_SUCCESS;
}

// Valida los argumentos y devuelve el tamaño de la matrices a crear
int validateArgs(int& argc, char* argv[])
{
    if (argc != 2)
    {
        std::cout << "Cantidad de argumentos incorrectos" << std::endl;
        exit(EXIT_FAILURE);
    }

    int size = atoi(argv[1]);
    if (size < MIN_MATRIX_SIZE || size > MAX_MATRIX_SIZE)
    {
        std::cout << "El tamaño de la matriz debe estar entre "
            << MIN_MATRIX_SIZE << " y " << MAX_MATRIX_SIZE << std::endl;

        exit(EXIT_FAILURE);
    }

    return size;
}

// Genera una matriz de igual tamaño de filas y columnas
Matrix matrixGenerator(int size, bool populate)
{
    // "Movemos" al objeto en vez de realizar una copia
    return std::move(matrixGenerator(size, size, populate));
}

// Genera una matriz y las llena de datos de ser necesario
Matrix matrixGenerator(int row_size, int col_size, bool populate)
{
    Matrix aux(row_size, std::vector<t_matrix>(col_size));

    if (populate)
    {
        std::random_device rd; // Obtenemos un número random del hardware
        std::mt19937 gen(rd()); // Generamos la semilla (seed)
        std::uniform_int_distribution<> distr(MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE); // Definimos el rango

        for (int i=0; i < row_size; ++i)
            for (int j=0; j < col_size; ++j)
                aux[i][j] = distr(gen);
    }

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Realiza la multiplicación entre 2 matrices y devuelve una nueva de manera secuencial
Matrix secuentialMult(const Matrix& a, const Matrix& b)
{
    Matrix aux = matrixGenerator(a.size(), false);

    for (int i=0; i < a.size(); ++i)
        for (int j=0; j < b[0].size(); ++j)
            for (int k=0; k < a.size(); ++k)
                aux[i][j] += a[i][k] * b[k][j];    

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Realiza la multiplicación entre 2 matrices y devuelve una nueva de manera concurrente
Matrix concurrentMult(const Matrix& a, const Matrix& b)
{
    Matrix aux = matrixGenerator(a.size(), false);
    std::vector<std::thread> workers;

    for (int i; i < a.size(); ++i)
    {
        workers.push_back(std::thread(
                // Función anónima o lambda que realiza la asignación y multiplicación
                [&aux, &a, &b, i]()
                {
                    for (int j=0; j < b[0].size(); ++j)
                        for (int k=0; k < a.size(); ++k)
                            aux[i][j] += a[i][k] * b[k][j];
                }
            )
        );
    }
    
    for(auto& worker : workers)
    {
        worker.join();
    }

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Imprime la matriz con su alias
void printMatrix(const std::string alias, const Matrix& m)
{
    std::cout << "Matrix " << alias << ":" << std::endl;
    
    for(const auto& row : m)
    {
        for(const auto& col : row)
        {
            std::cout << "\t" << col << "\t";
        }
        std::cout << std::endl;
    }
}


Overwriting matrix_mult.cpp


Compilamos el programa, indicando que utilice el módulo pthread para poder utilizar hilos

In [24]:
!g++ matrix_mult.cpp -pthread -std=c++14 -o matrix_mult.out

## Pruebas de validación de parametros:

In [25]:
!./matrix_mult.out

Cantidad de argumentos incorrectos


In [26]:
!./matrix_mult.out 1

El tamaño de la matriz debe estar entre 5 y 20


In [27]:
!./matrix_mult.out 21

El tamaño de la matriz debe estar entre 5 y 20


## Ejecución exitosa:

In [28]:
!./matrix_mult.out 10

Matrix A:
	-3		25		18		-25		1		-6		-27		17		-27		-5	
	-26		-20		19		12		25		-4		-9		-31		-15		18	
	9		15		31		-1		16		-27		2		25		13		-1	
	-31		1		29		24		-12		20		-7		8		30		11	
	28		-18		-17		-13		-13		11		20		20		18		-13	
	12		-23		25		-3		-11		8		-15		26		21		-32	
	14		-31		-22		15		-15		-26		11		-27		6		-1	
	17		-16		25		-4		0		-15		-21		-27		31		-1	
	14		-23		5		25		20		2		-6		26		1		32	
	26		11		3		6		-11		-19		-29		-6		-12		-3	
Matrix B:
	-25		-30		24		-8		16		19		-14		-31		21		-30	
	-21		-1		5		27		-25		12		7		20		6		1	
	2		-30		22		32		-17		-1		26		26		-19		-31	
	-19		-22		-1		26		-7		-25		24		-29		6		-10	
	-20		-19		14		-28		-6		28		-21		-3		1		-23	
	-3		7		-11		26		27		-15		27		27		-14		16	
	-1		-5		22		26		-21		-28		-26		6		-14		-30	
	1		5		31		-14		8		19		30		-23		16		22	
	-3		2		-3		4		31		-23		-32		27		1		-31	
	-4		6		-21		8		-6		9		3		9		27		-24	
Matrix CS:
	204		150		673		-647		-1076		2623		1963		294		168		1829	
	343		-569		-1416		68		-1213		-123		211		785		-773		-967